<a href="https://colab.research.google.com/github/donald-ye/Math-Stuff/blob/main/Solving_System_of_Linear_Equations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Team Project 3 - Solving a System of Linear Equations

In this project, we compare the performance of three different methods of solving a system of linear equations. The lesson that you may get is that finding an efficient method is not an unnecessary complication.

For this project, <b>DO NOT</b> use the 'solve' or 'inv' method in the linear algebra package! You have to create your own code for it.

1. (5 pts) Create a function **randmat(n)** which returns a random square matrix constructed as the following recipe.
<ul>
    <li>The size of the matrix is $n \times n$.</li>
    <li>Each off-diagonal entry ($a_{ij}$ where $i \ne j$) is a random number in $[0, 1)$. A random number can be constructed by the random method (see <a href="https://docs.scipy.org/doc/numpy/reference/routines.random.html">here</a>).</li>
    <li>A diagonal entry $a_{ii}$ is a random number in $[n, n+1)$. (This condition guarantees that the matrix $(a_{ij})$ is strictly diagonally dominant, hence invertible.)</li>
</ul>

And create a function **randvec(n)** which returns an $n$-dimensional random vector whose entries are random numbers in $[0, 100)$.

In [1]:
import numpy as np

def randvec(n):
  return np.random.uniform(0,100, size=(n,n))

def randmat(n):
  matrix = np.random.random((n,n))
  np.fill_diagonal(matrix, np.random.uniform(n, n+1, size=n))
  for i in range(n):
    for j in range(n):
        if i != j:
          matrix[i][j] = np.random.random()

  return matrix

n = int(input("Enter the n-dimensional size of your matrix: "))

print("Random matrix: \n",randmat(n)) # Diagonals = (n, n+1), non-Diagonals = random[0,1)
print("Random vector: \n",randvec(n))


Enter the n-dimensional size of your matrix: 3
Random matrix: 
 [[3.8604345  0.96462443 0.37677156]
 [0.92613082 3.9621313  0.09390755]
 [0.39545587 0.1938276  3.63920029]]
Random vector: 
 [[73.38432062 15.13751356 58.07652283]
 [32.53952245 67.52383197 43.46959309]
 [79.02926344 62.70858022 27.27785992]]


2. (10 pts) Create a function **GaussElim(A, b)** which solves a system of linear equations $Ax = b$ by using Gaussian Elimination with the partial pivoting.

In [11]:
def GaussElim(A,b):
  n = len(b)
  A = A.astype(float)
  b = b.astype(float)

  aug_matrix = np.hstack((A, b.reshape(n, 1)))

  for i in range(n):
    max_row = np.argmax(np.abs(aug_matrix[i:, i])) + i

    if i != max_row:
      aug_matrix[[i, max_row]] = aug_matrix[[max_row, i]]

    # Checks if pivot is zero, executes faster than (np.linalg.det(A) = 0)
    if np.isclose(aug_matrix[i, i], 0):
      print("Singular matrix detected. No unique solution exists.")
      return None

    aug_matrix[i] = aug_matrix[i] / aug_matrix[i, i]
    for j in range(i + 1, n):
      aug_matrix[j] -= aug_matrix[j, i] * aug_matrix[i]

  x = np.zeros(n)
  for i in range(n - 1, -1, -1):
    x[i] = aug_matrix[i, -1] - np.dot(aug_matrix[i, i+1:n], x[i+1:])

  return x

A = np.array([[2, 1, -1],
              [3, 2, 2],
              [1, 1, 1]], dtype=float)

b = np.array([4, 10, 3], dtype=float)
x = GaussElim(A,b)


print("Solution of vector x:", x)

# # Singular Matrix test case

# A = np.array([[2, 1, -1],
#               [4, 2, -2],
#               [1, 1, 1]], dtype=float)  # Second row is 2 * first row

# b = np.array([4, 8, 3], dtype=float)
# print(GaussElim(A, b))

Solution of vector x: [ 4.  -2.5  1.5]
Singular matrix detected. No unique solution exists.
None


3. (10 pts) Create a function **InvMat(A,b)** which solves a system of linear equations $Ax = b$ with the "theoretically simplest method," that is, computing $x = A^{-1}b$. Compute the inverse matrix as the following:
<ul>
    <li>Make an augmented matrix $[A | I]$ where $I$ is the $n \times n$ identity matrix.</li>
    <li>Apply elementary row operations until the left half $A$ on $[A| I]$ becomes $I$, so it looks $[I | B]$.</li>
    <li>Then the right half of the augmented matrix $B$ is $A^{-1}$.</li>
</ul>

In [12]:
def InvMat(A,b):
  n = len(A)
  A = A.astype(float)
  b = b.astype(float)

  aug_matrix = np.hstack((A, np.identity(n)))

  # Check if A is square
  if A.shape[0] != A.shape[1]:
      print("Non-Square matrix detected, matrix can not be inverted.")
      return None

  # Check if A is invertible
  if np.linalg.det(A) == 0:
      print("Singular matrix detecetd, matrix can not be inverted.") # singular matrix <=> det(A) = 0
      return None

  for i in range(n):
    aug_matrix[i] = aug_matrix[i] / aug_matrix[i, i]
    for j in range(i+1, n):
        aug_matrix[j] -= aug_matrix[j, i] * aug_matrix[i]

  for i in range(n-1, -1, -1):
    for j in range(i-1, -1, -1):
      aug_matrix[j] -= aug_matrix[j, i] * aug_matrix[i]

  # Inverse and solution
  A_inv = aug_matrix[:, n:]
  x = np.dot(A_inv, b)
  return x

A = np.array([[2, 1, -1],
              [3, 2, 2],
              [1, 1, 1]], dtype=float)
b = np.array([4, 10, 3], dtype=float)
x = InvMat(A,b)

print("In the format Ax=b, given known values of \nA= \n", A ,"and \nb= \n",b ,"\nUtilizing the inverse matrix, we conclude that\nx =\n ", x)

# # Non-zero matrix test case
# A = np.array([
#     [2, 1],
#     [4, 2],
#     [1, 3]
# ], dtype=float)

# b = np.array([5, 10, 6], dtype=float)
# print(InvMat(A,b))

# # Singular matrix test case
# A = np.array([
#     [1, 2, 3],
#     [2, 4, 6],
#     [1, 1, 1]
# ], dtype=float)

# b = np.array([6, 12, 3], dtype=float)
# print(InvMat(A,b))

In the format Ax=b, given known values of 
A= 
 [[ 2.  1. -1.]
 [ 3.  2.  2.]
 [ 1.  1.  1.]] and 
b= 
 [ 4. 10.  3.] 
Utilizing the inverse matrix, we conclude that
x =
  [ 4.  -2.5  1.5]


4. (10 pts) Create a function **Jacobi(A, b, err)** which solves a system of linear equations $Ax = b$ by using Jacobi interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

In [45]:
def Jacobi(A, b, err):
  n = len(A)
  A = A.astype(float)
  b = b.astype(float)

  # Check if A is invertible
  if np.linalg.det(A) == 0:
      print("Singular matrix detected, Jacobi Iteration Method can not be applied.") # singular matrix <=> det(A) = 0
      return None

  x_old = np.zeros(n)
  x_new = np.zeros(n)

  k = 1000
  for i in range(k):
    for j in range(n):
      sum = 0
      for k in range(n):
        if j != k:
          sum += A[j,k] * x_old[k]

      x_new[j] = (b[j] - sum) / A[j,j] # update

    #infinity norm computation
    max_diff = 0
    for i in range(n):
      diff = abs(x_new[i] - x_old[i])
      if diff > max_diff:
        max_diff = diff

    if max_diff < err:
      return x_new

    #copy
    for i in range(n):
      x_old[i] = x_new[i]

  # break when > err || k =
  print("Jacobi method did not converge within 1000 iterations.")
  return x_new

A = np.array([[2, 1, -1],
              [3, 2, 2],
              [1, 1, 1]], dtype=float)
b = np.array([4, 10, 3], dtype=float)
err = 1e-7

x = Jacobi(A, b, err)

print("In the format Ax=b, given known values of \nA= \n", A ,"and \nb= \n",b ,"\nUtilizing the Jacobi Iteration Method, we conclude that\nx =\n ", x)


In the format Ax=b, given known values of 
A= 
 [[ 2.  1. -1.]
 [ 3.  2.  2.]
 [ 1.  1.  1.]] and 
b= 
 [ 4. 10.  3.] 
Utilizing the Jacobi Iteration Method, we conclude that
x =
  [-inf  inf  nan]


<ipython-input-45-f5e3269cea3d>:20: RuntimeWarning: overflow encountered in scalar add
  sum += A[j,k] * x_old[k]
<ipython-input-45-f5e3269cea3d>:27: RuntimeWarning: invalid value encountered in scalar subtract
  diff = abs(x_new[i] - x_old[i])
<ipython-input-45-f5e3269cea3d>:20: RuntimeWarning: invalid value encountered in scalar add
  sum += A[j,k] * x_old[k]


5. (10 pts) Create a function **GaussSeidel(A, b, err)** which solves a system of linear equations $Ax = b$ by using Gauss-Seidel interation method. Set $x^{(0)} = \vec{0}$. We stop the iteration when the estimation of the error $||x^{(k)} - x^{(k-1)}||_\infty$ is less than err or $k = 1000$. (Here $x^{(k)}$ is the $k$-th output of the iteration).

6. (5 pts) For $n = 100, 200, 300, \cdots , 1000$, create a random $n \times n$ matrix $A$ and a random $n$-dimensional vector $b$. Solve the system of linear equations $Ax = b$ by using **GaussElim(A, b)**, **InvMat(A, b)**, **Jacobi(A, b, err)**, and **GaussSeidel(A, b, err)**. Use $10^{-6}$ for the error tolerance. Record the excution time for each method. Plot the graph of the excution time for those three methods on the same plane.

For the computation of the excution time, you may use the following method:

In [ ]:
import time

start = time.time()
"the code you want to test stays here"
end = time.time()

print(end - start)